In [1]:
import yaml
from pprint import pprint
from copy import deepcopy

In [2]:
config_filename1 = "/Users/lucas/Projects/config1"
config_filename2 = "/Users/lucas/Projects/config2"

In [3]:
with open(config_filename1) as f:
    yaml1 = yaml.load(f, Loader=yaml.FullLoader)
    
with open(config_filename2) as f:
    yaml2 = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
class Config:
    
    _keys_str = {
        "apiVersion",
        "current-context",
        "kind"
    }
    
    _keys_list = {
        "clusters",
        "contexts",
        "users"
    }
        
    @staticmethod
    def verify(config):
        return False not in [k in config.get_content().keys() for k in Config._keys_str.union(Config._keys_list)]
    
    def __init__(self, config=None):
        self._content = {}
        if config is not None:
            self._content = deepcopy(config)
            
    def is_empty(self):
        return self._content == {}
    
    def get_content(self):
        return self._content
            
    def __add__(self, config):
        if self.is_empty():
            return deepcopy(config.get_content())
        
        if config.is_empty():
            return self.get_content()
        
        assert Config.verify(self) == True
        assert Config.verify(config) == True
        
        assert self._content['kind'] == 'Config'
        assert config.get_content()['kind'] == 'Config'
        
        assert self._content['apiVersion'] == config.get_content()['apiVersion']
        
        merged = {k:self._content[k]+config.get_content()[k] for k in Config._keys_list}
        
        merged['kind'] = 'Config'
        merged['apiVersion'] = self._content['apiVersion']
        merged['current-context'] = self._content['current-context']
        
        return Config(deepcopy(merged))
            
    
    def __iadd__(self, config):
        if self._content == {}:
            self._content = deepcopy(config.get_content())
        else:
            self._content = self.__add__(config)

In [5]:
config1 = Config(yaml1)
config2 = Config(yaml2)

In [6]:
new_config = config1 + config2

In [7]:
output_filename = "/Users/lucas/Projects/config"

with open(output_filename, 'w') as f:
    f.write(
        yaml.dump(new_config.get_content())
    )